In [4]:
# Importações
import pandas as pd # Importando o pandas para que eu possa mexer com as planilhas de uma forma mais organizada, também instalei o openpyxl para ler arquivos.
from selenium import webdriver
from selenium.webdriver.common.by import By # importando o by do webdriver para puder utilizar a escolha de seletor css, id, class, etc
from selenium.webdriver.common.keys import Keys
import time # importando o time para fazer uma pause enquanto carrega dados da planilha ou carrega o site
import re # Importando o re para puder utilizar o regex e fazer o tratamento das entradas que vem da planilha


In [5]:
# Configurações de navegação e login
navegador = webdriver.Chrome() # Escolhendo qual será o browser que vai realizaro o processo

url = "https://projetos.ati.pe.gov.br/redmine/issues/"
navegador.get(url) # Dizendo qual será o site que vai ser realizada a otimização

# Realizando o login
try:
    navegador.find_element(By.ID, "username").send_keys("pablo.henrique1") # Obtendo, escrevendo o usuário
    navegador.find_element(By.ID, "password").send_keys("@Tais84671514") # Obtendo, escrevendo a senha
    navegador.find_element(By.CSS_SELECTOR, 'input[type="submit"]').click() # Obtendo o input submit e realizando o click
    time.sleep(2)
except Exception as e:
    print(f"Erro no login: {e}") # Tratando as excessões

In [6]:
# Caminho do arquivo
arquivo = "C:/Users/devpa/Downloads/bancoredminescripts.xlsx"

# Carregar a planilha e converter tudo para string para evitar truncamentos
dataFrame = pd.read_excel(arquivo, dtype=str)

# Remover espaços extras dos nomes das colunas
dataFrame.columns = dataFrame.columns.str.strip()

# Garantir que as colunas existem no DataFrame
colunas_validas = ['registrado no redmine', 'nao registrado no redmine']
for coluna in colunas_validas:
    if coluna not in dataFrame.columns:
        raise ValueError(f"A coluna '{coluna}' não foi encontrada no arquivo.")

# Remover espaços extras nos valores das colunas
for coluna in colunas_validas:
    dataFrame[coluna] = dataFrame[coluna].str.strip()  # Remove espaços extras
    dataFrame[coluna] = dataFrame[coluna].str.replace(r'\.0$', '', regex=True)  # Remove ".0"
    dataFrame[coluna] = dataFrame[coluna].fillna('')  # Substitui NaN por string vazia

# Expressão regular para aceitar números de até 6 dígitos
regex = r'^\d{1,6}$'

# Filtrar os dados corretamente
dataFrame_filtrado = dataFrame[
    dataFrame['registrado no redmine'].astype(str).str.match(regex, na=False) |
    dataFrame['nao registrado no redmine'].astype(str).str.match(regex, na=False)
]

# Criando dicionário para armazenar os valores
processosEScripts = {}

# Iterando sobre as colunas de interesse
for coluna in ["registrado no redmine", "nao registrado no redmine"]:
    for valor in dataFrame_filtrado[coluna]:
        inputBuscar = navegador.find_element(By.XPATH, "//input[@id='q']")
        inputBuscar # Obtém o campo de pesquisa
        inputBuscar.clear()  # Limpa o campo antes de digitar
        inputBuscar.send_keys(valor)  # Digita o valor da iteração
        inputBuscar.submit()  # Executa a busca
        
        time.sleep(1)  # Aguarda a página carregar
        
        try:
            # Obtém a informação do script na página
            containerDiv = navegador.find_element(By.CLASS_NAME, 'cf_60')
            appScript = containerDiv.find_element(By.CLASS_NAME, 'value')
            simNao = appScript.text.strip()  # Remove espaços extras

            # Salva no dicionário
            processosEScripts[valor] = simNao
            print(f"Processo {valor}: {simNao}")  
        
        except Exception as e:
            print(f"Erro ao buscar processo {valor}: {e}")
            processosEScripts[valor] = "Erro"

# Criando colunas "scripts" e "scripts_2" caso não existam
if "scripts" not in dataFrame_filtrado.columns:
    dataFrame_filtrado.insert(dataFrame_filtrado.columns.get_loc("registrado no redmine") + 1, "scripts", "")

if "scripts_2" not in dataFrame_filtrado.columns:
    dataFrame_filtrado.insert(dataFrame_filtrado.columns.get_loc("nao registrado no redmine") + 1, "scripts_2", "")

# Atualizando as colunas "scripts" e "scripts_2" com os valores do dicionário
dataFrame_filtrado["scripts"] = dataFrame_filtrado["registrado no redmine"].map(processosEScripts)
dataFrame_filtrado["scripts_2"] = dataFrame_filtrado["nao registrado no redmine"].map(processosEScripts)

# Salvando a planilha original com as alterações
dataFrame_filtrado.to_excel(arquivo, index=False)
print(f"Planilha atualizada salva em {arquivo}")

Processo 170997: Sim
Processo 170821: Sim
Processo 170661: Sim
Processo 170616: Sim
Processo 170614: Sim
Processo 170546: Sim
Processo 170539: Sim
Processo 170532: Sim
Processo 170501: Sim
Processo 170446: Sim
Processo 170442: Sim
Processo 170441: Sim
Processo 170432: Sim
Processo 170412: Sim
Processo 170400: Sim
Processo 170341: Sim
Processo 170329: Sim
Processo 170325: Sim
Processo 170321: Sim
Processo 170319: Sim
Processo 170303: Sim
Processo 170280: Sim
Processo 170278: Sim
Processo 170262: Sim
Processo 170215: Sim
Processo 170189: Sim
Processo 170185: Sim
Processo 170169: Sim
Processo 170081: Sim
Processo 169946: Sim
Processo 169855: Sim
Processo 169849: Sim
Processo 169840: Sim
Processo 169755: Sim
Processo 169751: Sim
Processo 169747: Sim
Processo 169681: Sim
Processo 169647: Sim
Processo 169643: Sim
Processo 169641: Sim
Processo 169588: Sim
Processo 169580: Sim
Processo 169577: Sim
Processo 169563: Sim
Processo 169473: Sim
Processo 169349: Sim
Processo 169283: Sim
Processo 1692

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=55608): Read timed out. (read timeout=120)